<style>
    body, * {
        direction: rtl !important;
        text-align: right !important;
    }
</style>
מקור: [Spark By Examples - PySpark Repartition vs Coalesce](https://sparkbyexamples.com/pyspark/pyspark-repartition-vs-coalesce/)

# 🔄 PySpark: Repartition() לעומת Coalesce()

## 📚 מבוא

ב-PySpark, הבחירה בין הפונקציות `repartition()` ו-`coalesce()` חשובה ביותר לאופטימיזציה של ביצועים וניצול משאבים.

שיטות אלו ממלאות תפקידים מרכזיים בארגון מחדש של מחיצות (partitions) בתוך DataFrame, אך הן שונות במנגנוני הפעולה שלהן והשלכותיהן.

### 🎯 נקודות מפתח:

**במילים פשוטות:** `repartition()` מגדילה או מקטינה את המחיצות, בעוד ש-`coalesce()` רק מקטינה את מספר המחיצות בצורה יעילה.

במאמר זה נלמד את ההבדלים בין repartition ל-coalesce של PySpark עם דוגמאות מעשיות.

### 📋 תוכן עניינים:

1. **RDD Partition**
   - RDD repartition
   - RDD coalesce

2. **DataFrame Partition**
   - DataFrame repartition
   - DataFrame coalesce

---

### ⚠️ הערה חשובה:

נקודה חשובה לציין היא ש-`repartition()` ו-`coalesce()` של PySpark הן **פעולות יקרות מאוד** מכיוון שהן **מערבבות את הנתונים על פני מחיצות רבות**. לכן, יש לצמצם את השימוש בהן ככל האפשר.

---

## 1️⃣ PySpark RDD: Repartition() לעומת Coalesce()

ניצור RDD עם מחיצות ונשתמש בו כדי להדגים את `repartition()` ו-`coalesce()`

### 🔧 יצירת Spark Session והכנת נתונים

In [13]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SparkByExamples.com").getOrCreate()

# יצירת spark session עם localhost
rdd = spark.sparkContext.parallelize(range(0,20))
print("From local[5] : " + str(rdd.getNumPartitions()))

From local[5] : 8


In [14]:
# Create spark session with local[5]
rdd = spark.sparkContext.parallelize(range(0,20))
print("From local[5] : "+str(rdd.getNumPartitions()))

# Use parallelize with 6 partitions
rdd1 = spark.sparkContext.parallelize(range(0,25), 6)
print("parallelize : "+str(rdd1.getNumPartitions()))

From local[5] : 8
parallelize : 6


In [15]:
file_path = "tmp/test.txt"

import os
os.makedirs(os.path.dirname(file_path), exist_ok=True)

with open(file_path, "w", encoding="utf-8") as f:
    f.write("Hello world\n")
    f.write("Python and Spark\n")
    f.write("RDD and DataFrame\n")

# קריאת קובץ טקסט
rddFromFile = spark.sparkContext.textFile(file_path, 10)
print("TextFile : " + str(rddFromFile.getNumPartitions()))

TextFile : 12




השיטה `sparkContext.parallelize()` ב-PySpark משמשת להקבלה (parallelize) של אוסף לתוך RDD מבוזר עמיד (Resilient Distributed Dataset). בדוגמה הנתונה, `Range(0,20)` יוצר טווח מספרים מ-0 עד 19 (כולל). הארגומנט השני, 6, מציין את מספר המחיצות שהנתונים צריכים להיחלק אליהן.

בואו נכתוב זאת לקובץ ונבדוק את הנתונים. שימו לב שהמחיצות שלכם עשויות להכיל רשומות שונות.

In [16]:
rdd1.saveAsTextFile("tmp/partition")

זה מייצר קבצים של 5 חלקים (part files), אחד לכל מחיצה:

```
Partition 1 : 0 1 2
Partition 2 : 3 4 5
Partition 3 : 6 7 8 9
Partition 4 : 10 11 12
Partition 5 : 13 14 15
Partition 6 : 16 17 18 19
```

---

### 1.1 📊 RDD repartition()

`repartition()` היא שיטת טרנספורמציה זמינה ב-RDDs (Resilient Distributed Datasets) שמחלקת מחדש נתונים על פני מספר מוגדר של מחיצות.

כאשר אתם קוראים `repartition(n)`, כאשר n הוא מספר המחיצות הרצוי, Spark מערבב מחדש את הנתונים ב-RDD לתוך בדיוק n מחיצות.

אם אתם מגדילים/מקטינים את מספר המחיצות באמצעות `repartition()`, Spark יבצע ערבוב מלא (full shuffle) של הנתונים על פני הצומת (cluster), שיכול להיות פעולה יקרה, במיוחד עבור מערכי נתונים גדולים.

In [17]:
# שימוש ב-repartition
rdd2 = rdd1.repartition(4)
print("Repartition size : " + str(rdd2.getNumPartitions()))
rdd2.saveAsTextFile("tmp/re-partition")

Repartition size : 4


התוצאה מציגה "גודל Repartition" של 4, המצביעה על כך שהנתונים חולקו מחדש על פני 4 מחיצות.

פעולה זו כוללת ערבוב מלא, שיכול להיות די יקר, במיוחד כאשר מטפלים במערכי נתונים גדולים ביותר במיליארדים או טריליונים.

**פלט:**

```
Partition 1 : 1 6 10 15 19
Partition 2 : 2 3 7 11 16
Partition 3 : 4 8 12 13 17
Partition 4 : 0 5 9 14 18
```

---

### 1.2 🔽 RDD coalesce()

ב-PySpark, `coalesce()` היא שיטת טרנספורמציה זמינה ב-RDDs (Resilient Distributed Datasets) שמפחיתה את מספר המחיצות **ללא ערבוב** נתונים על פני הצומת.

כאשר אתם קוראים `coalesce(n)`, כאשר n הוא מספר המחיצות הרצוי, Spark ממזג מחיצות קיימות כדי ליצור n מחיצות.

In [18]:
# שימוש ב-coalesce()
rdd3 = rdd1.coalesce(4)
print("Repartition size : " + str(rdd3.getNumPartitions()))
rdd3.saveAsTextFile("tmp/coalesce")

Repartition size : 4


אם נשווה את הפלט הבא עם סעיף 1.1, יתברר שמחיצה 3 הועברה למחיצה 2, ומחיצה 6 הועברה למחיצה 5.

תנועת נתונים זו התרחשה רק בין שתי מחיצות.

**פלט:**

```
Partition 1 : 0 1 2
Partition 2 : 3 4 5 6 7 8 9
Partition 4 : 10 11 12
Partition 5 : 13 14 15 16 17 18 19
```

---

## 2️⃣ PySpark DataFrame: repartition() לעומת coalesce()

בדומה ל-RDD, אינכם יכולים לציין את המחיצה/הקבלה בעת יצירת DataFrame.

DataFrame כברירת מחדל משתמש באופן פנימי בשיטות המפורטות בסעיף 1 כדי לקבוע את המחיצה ברירת המחדל ומחלק את הנתונים לפרלליזם.

אם אינכם מכירים את DataFrame, אמליץ ללמוד אותו על ידי ביקור ב[PySpark DataFrame Tutorial](https://sparkbyexamples.com/pyspark-tutorial/) לפני שתמשיכו במאמר זה.

### 🔧 דוגמת DataFrame

In [19]:
# דוגמת DataFrame
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('SparkByExamples.com') \
    .master("local[5]").getOrCreate()

df = spark.range(0,20)
print(df.rdd.getNumPartitions())

df.write.mode("overwrite").csv("tmp/partition.csv")

8


הדוגמה לעיל יוצרת 5 מחיצות כפי שצוין ב-`master("local[5]")` והנתונים מופצים על פני כל 5 המחיצות.

**פלט:**

```
Partition 1 : 0 1 2 3
Partition 2 : 4 5 6 7
Partition 3 : 8 9 10 11
Partition 4 : 12 13 14 15
Partition 5 : 16 17 18 19
```

---

### 2.1 📈 DataFrame repartition()

השיטה `repartition()` של DataFrame ב-PySpark מחלקה מחדש (מגדילה או מקטינה מחיצות) נתונים באופן שווה על פני מספר מוגדר של מחיצות, תוך אופטימיזציה של פרלליזם וניצול משאבים.

היא מפעילה ערבוב מלא של נתונים ושימושית להתאמת סכימת החלוקה למחיצות עבור פעולות במורד הזרם כמו joins ואגרגציות.

דוגמה זו מרחיבה את מספר המחיצות מ-5 ל-6 על ידי חלוקה מחדש של נתונים על פני כל המחיצות.

In [20]:
# DataFrame repartition
df2 = df.repartition(6)
print(df2.rdd.getNumPartitions())

6


רק הגדלת מחיצה 1 גורמת לתנועות נתונים מכל המחיצות.

**פלט:**

```
Partition 1 : 14 3 5
Partition 2 : 4 16 15
Partition 3 : 8 3 18
Partition 4 : 12 2 19
Partition 5 : 6 17 7 0
Partition 6 : 9 10 11 13
```

וגם, הקטנת המחיצות גם גורמת לתנועת נתונים מכל המחיצות.

לכן כאשר רציתם להקטין את ההמלצה של המחיצה היא להשתמש ב-`coalesce()`

---

### 2.2 🔽 DataFrame coalesce()

Spark DataFrame `coalesce()` משמש רק להקטנת מספר המחיצות. זוהי גרסה משופרת או אופטימלית של repartition() שבה תנועת הנתונים על פני המחיצות פחותה באמצעות coalesce.

In [21]:
# DataFrame coalesce
df3 = df.coalesce(2)
print(df3.rdd.getNumPartitions())

2


זה מניב פלט 2 והמחיצה המתקבלת נראית כך:

**פלט:**

```
Partition 1 : 0 1 2 3 8 9 10 11
Partition 2 : 4 5 6 7 12 13 14 15 16 17 18 19
```

מכיוון שאנחנו מצמצמים 5 ל-2 מחיצות, תנועת הנתונים מתרחשת רק מ-3 מחיצות והיא עוברת לנשאר 2 מחיצות.

---

## 3️⃣ Default Shuffle Partition

ב-PySpark, מחיצת הערבוב (shuffle partition) ברירת המחדל מתייחסת למספר המחיצות ש-Spark משתמש בהן בעת ביצוע פעולות ערבוב, כגון joins, group-bys ואגרגציות.

פעולות ערבוב כוללות חלוקה מחדש של נתונים על פני צמתים שונים בצומת, שיכולה להיות יקרה מבחינת חישוב ולהשפיע על הביצועים.

כברירת מחדל, Spark מגדיר את מספר מחיצות הערבוב ל-**200**.

ערך ברירת מחדל זה נשלט על ידי פרמטר התצורה `spark.sql.shuffle.partitions`.

אתם יכולים להתאים הגדרה זו על סמך גודל הנתונים שלכם והמשאבים של הצומת שלכם כדי לייעל את הביצועים.

In [22]:
# ספירת מחיצות ברירת מחדל
df4 = df.groupBy("id").count()
print(df4.rdd.getNumPartitions())

8


לאחר פעולות ערבוב, אתם יכולים לשנות את המחיצות באמצעות coalesce() או repartition().

---

## 4️⃣ PySpark: repartition לעומת coalesce - טבלת השוואה

להלן ההבדלים בפורמט טבלה:

| **תכונה** | **REPARTITION** | **COALESCE** |
|------------|-----------------|---------------|
| **תיאור** | מתאים את מספר המחיצות, מחלק מחדש נתונים על פני המספר המוגדר של מחיצות. | מפחית את מספר המחיצות מבלי לערבב נתונים, ממזג מחיצות קיימות. |
| **ערבוב מלא** | כן | לא |
| **יקר מבחינת משאבים** | יכול להיות יקר, במיוחד עבור מערכי נתונים גדולים, מכיוון שהוא כולל ערבוב מלא של נתונים. | פחות יקר מ-repartition, מכיוון שהוא ממזער תנועת נתונים על ידי שילוב מחיצות רק כשאפשר. |
| **תנועת נתונים** | מחלק נתונים על פני מחיצות באופן שווה, עלול להוביל לגדלי מחיצות מאוזנים. | עלול לגרום לגדלי מחיצות לא מאוזנים, במיוחד בעת הפחתת מספר המחיצות. |
| **מקרי שימוש** | שימושי בעת שינוי מספר המחיצות או חלוקה שווה של נתונים על פני מחיצות. | שימושי בעת הקטנת מספר המחיצות מבלי לשאת בעלות של ערבוב מלא. |

---

## 🎯 סיכום

במאמר זה על PySpark repartition() לעומת coalesce(), למדתם:

- ✅ כיצד ליצור RDD עם מחיצות
- ✅ כיצד להחליק מחיצות ב-RDD באמצעות coalesce()
- ✅ כיצד לחלק מחדש DataFrame באמצעות repartition()
- ✅ כיצד לחלק מחדש DataFrame באמצעות coalesce()
- ✅ למדתם את ההבדל בין repartition ו-coalesce

---

## 📚 מאמרים קשורים

- [PySpark partitionBy() Explained with Examples](https://sparkbyexamples.com/pyspark/pyspark-partitionby-explained-with-examples/)
- [PySpark Parallelize | Create RDD](https://sparkbyexamples.com/pyspark/pyspark-parallelize-create-rdd/)
- [PySpark repartition() vs partitionBy() with Examples](https://sparkbyexamples.com/pyspark/pyspark-repartition-vs-partitionby-with-examples/)
- [PySpark repartition() – Explained with Examples](https://sparkbyexamples.com/pyspark/pyspark-repartition-explained-with-examples/)
- [PySpark repartition() vs partitionBy()](https://sparkbyexamples.com/pyspark/pyspark-repartition-vs-partitionby/)
- [PySpark Query Database Table using JDBC](https://sparkbyexamples.com/pyspark/pyspark-query-database-table-using-jdbc/)
- [PySpark Read and Write SQL Server Table](https://sparkbyexamples.com/pyspark/pyspark-read-and-write-sql-server-table/)
- [PySpark Read JDBC Table to DataFrame](https://sparkbyexamples.com/pyspark/pyspark-read-jdbc-table-to-dataframe/)

---

## 🔗 מקורות

- [Apache Spark Configuration Documentation](https://spark.apache.org/docs/latest/configuration.html)
- [Apache Spark RDD Programming Guide](https://spark.apache.org/docs/latest/rdd-programming-guide.html)

---

### 🎓 למידה מהנה!

**Happy Learning !!**

---

## 📝 הערות נוספות

### 💡 טיפים חשובים:

1. **השתמשו ב-coalesce() להקטנה**: כאשר אתם צריכים להקטין את מספר המחיצות, העדיפו תמיד את coalesce() על פני repartition() לביצועים טובים יותר.

2. **השתמשו ב-repartition() להגדלה או חלוקה שווה**: כאשר אתם צריכים להגדיל את מספר המחיצות או לחלק נתונים באופן שווה, השתמשו ב-repartition().

3. **שקלו את גודל הנתונים**: פעולות ערבוב יכולות להיות יקרות על מערכי נתונים גדולים. תכננו את אסטרטגיית החלוקה למחיצות שלכם בקפידה.

4. **התאימו את spark.sql.shuffle.partitions**: עבור אפליקציות ייצור, התאימו את הפרמטר הזה על סמך גודל הנתונים והמשאבים הזמינים של הצומת.

### ⚡ שיטות עבודה מומלצות:

- מזערו את כמות פעולות הערבוב באפליקציית Spark שלכם
- השתמשו בחלוקה למחיצות אופטימלית עבור מערכי הנתונים שלכם
- עקבו אחר מדדי הביצועים כדי לזהות צווארי בקבוק
- השתמשו ב-Spark UI כדי לנטר את פעולות הערבוב

---

### 📧 יצירת קשר ומשוב

אם יש לכם שאלות או משוב על מדריך זה, אנא בקרו ב-[SparkByExamples.com](https://sparkbyexamples.com)

---

**תאריך יצירה:** יולי 2023  
**תורגם ועובד לעברית:** נובמבר 2025  
**גרסה:** 1.0

In [23]:
import shutil

# מחיקה של כל התקייה /tmp
shutil.rmtree("tmp", ignore_errors=True)